In [54]:
import pandas as pd
df = pd.read_csv('D:/Project-Fake_news_classifier/data/processed/fake_news_processed.csv')

In [55]:
df.head()

,text,label,processed_text
0,no comment is expected from barack obama membe...,1,no comment is expected from barack obama membe...
1,did they post their votes for hillary already,1,did they post their vote for hillary already
2,now most of the demonstrators gathered last n...,1,now most of the demonstrator gathered last nig...
3,a dozen politically active pastors came here f...,0,a dozen politically active pastor came here fo...
4,the rs28 sarmat missile dubbed satan 2 will re...,1,the rs28 sarmat missile dubbed satan 2 will re...


In [56]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [57]:
df['text'] = df['text'].astype(str)

In [70]:
tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")
tokenizer.fit_on_texts(df['text'])
vocab_size = len(tokenizer.word_index)

In [71]:
vocab_size = min(10000, len(tokenizer.word_index) + 1)

In [72]:
sequences = tokenizer.texts_to_sequences(df['text'])
sequence_lengths = [len(seq) for seq in sequences]

In [73]:
max_length = max(sequence_lengths)

In [74]:
padded_text = pad_sequences(sequences,maxlen = 2000, padding='post', truncating='post')

print(padded_text.shape)

(62718, 2000)


In [75]:
from sklearn.model_selection import train_test_split


# Replace `labels` with your actual labels for the dataset
labels = df['label'].values  # Use the actual column for labels

# Split the data
X_train, X_test, y_train, y_test = train_test_split(padded_text, labels, test_size=0.2, random_state=42)


In [76]:
print('the shape of X_train:',X_train.shape)
print('the shape of y_train:',y_train.shape)


the shape of X_train: (50174, 2000)
the shape of y_train: (50174,)


In [77]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Build the LSTM model
model = Sequential()

# Embedding layer (size of input sequence is 2000)
model.add(Embedding(input_dim=vocab_size, output_dim=128, input_length=2000))

# LSTM layer
model.add(LSTM(units=64, dropout=0.2, recurrent_dropout=0.2))

# Fully connected layer with sigmoid activation for binary classification
model.add(Dense(units=1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])

# Model Summary
model.summary()


Model: "sequential_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_8 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_9 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Assuming the LSTM model is defined as in the previous step

# Train the model
history = model.fit(X_train, y_train, epochs=5, batch_size=64, validation_split=0.2)


test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=2)
print(f"Test Accuracy: {test_accuracy:.4f}")


Epoch 1/5
628/628 ━━━━━━━━━━━━━━━━━━━━ 1567s 2s/step - accuracy: 0.5471 - loss: 0.6896 - val_accuracy: 0.5570 - val_loss: 0.6868
Epoch 2/5
100/628 ━━━━━━━━━━━━━━━━━━━━ 27:40 3s/step - accuracy: 0.5555 - loss: 0.6875